In [1]:
from torchvision.models import feature_extraction
from torch import optim,nn
from sklearn import svm
import numpy as np
from torchvision import transforms
import torchvision.models as  models
from torch.utils.data import DataLoader
from src.dataset import PascalVOC_Dataset
import torch.optim as optim
from src.utils import encode_labels
import os
import torch.utils.model_zoo as model_zoo 
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import cv2
import torch
from PIL import Image


In [22]:
arr=[[0.5,0.5,0.6],[0.7,0.8,0.4]]
arr2=[[0.5,0.5,0.6],[0.7,0.8,0.4]]
arr=np.array(arr)
arr2=np.array(arr2)
concat=np.concatenate((arr,arr2),axis=0)

array([[0.5, 0.7, 0.5, 0.7],
       [0.5, 0.8, 0.5, 0.8],
       [0.6, 0.4, 0.6, 0.4]])

In [2]:
path='VOCdevkit-1/VOCdevkit/VOC2007/ImageSets/Main/'
img_dict=dict()
counter=0
for file in os.listdir(path):
    if(file ==f"train.txt"):
        file_obj=open(os.path.join(path,f"{file}"),"r")
        file_names=file_obj.read()
        file_names=file_names.splitlines()
        for file_name in file_names:
            
            img_dict[counter]=(file_name,0)
            counter+=1



In [10]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                     'cow', 'diningtable', 'dog', 'horse',
                     'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']

path='VOCdevkit-1/VOCdevkit/VOC2007/ImageSets/Main/'
counter=0
for object in object_categories:
    for file in os.listdir(path):
        name=str(object)+'_train.txt'
        if(file==name):
            file_obj=open(os.path.join(path,f"{file}"),"r")
            file_names=file_obj.read()
            lines=file_names.splitlines()
            for line in lines:
                content=line.split()
                if(('-1' or '0') not in content):
                    for key,value in img_dict.items():
                        if(value[0] ==content[0]):
                              img_dict[key]=(img_dict[key][0],counter)
    counter+=1                
    

            

In [3]:
img_path='VOCdevkit-1/VOCdevkit/VOC2007/JPEGImages/'
train_img=[]
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
train_transformation = transforms.Compose([
                                      transforms.Resize((224, 224)),                                      
                                      transforms.RandomChoice([
                                          transforms.ColorJitter(brightness=(0.80, 1.20)),
                                          transforms.RandomGrayscale(p = 0.25)
                                          ]),
                                      transforms.RandomHorizontalFlip(p = 0.25),
                                      transforms.RandomRotation(25),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean = mean, std = std),
                                      ])
valid_transformation=transforms.Compose([transforms.Resize((224,224)), 
                                          transforms.ToTensor(), 
                                           transforms.Normalize(mean = mean, std = std),
                                          ])

data_dir='./VOCdevkit-1'
# Create train dataloader
dataset_train = PascalVOC_Dataset(data_dir,
                                      year='2007', 
                                      image_set='train', 
                                      download=False, 
                                      transform=train_transformation, 
                                      target_transform=encode_labels)
    
train_loader = DataLoader(dataset_train, batch_size=16, num_workers=0, shuffle=True)
dataset_valid = PascalVOC_Dataset(data_dir,
                                      year='2007', 
                                      image_set='val', 
                                      download=False, 
                                      transform=valid_transformation, 
                                      target_transform=encode_labels)
    
valid_loader = DataLoader(dataset_valid, batch_size=16, num_workers=0)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   

model.fc = nn.Sequential(nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 20)).to(device)
model.load_state_dict(torch.load('models/Resnet50.h5'))
model


d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
model.fc[1]=nn.Identity()
model.fc[2]=nn.Identity()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
def feature_extractor(model,device,train_loader,valid_loader):
      train_feats=[]
      val_feats=[]
      
      with torch.no_grad():   
         
       for idx,(data, target) in enumerate(train_loader):
                    
            data, target = data.to(device), target.to(device)
                    
                    # zero the parameter gradients
            features = model(data)
            for feature in features:
                train_feats.append(feature.cpu().detach().numpy())
                        


       for idx,(data,target) in enumerate(valid_loader):
                data,target=data.to(device),target.to(device)
                
                features=model(data)
                for feature in features:
                  val_feats.append(feature.cpu().detach().numpy())

                    
                    
                    
               
                
      return train_feats,val_feats          
                        
                        
            


In [10]:
resnet50_train_feats,resnet50_val_feats=feature_extractor(model,device,train_loader,valid_loader)

In [11]:
resnet50_val_feats=np.array(resnet50_val_feats)
resnet50_train_feats=np.array(resnet50_train_feats)
np.savez("resnet50_train_feats.npz",resnet50_train_feats)
np.savez("resnet50_val_feats.npz",resnet50_val_feats)


(2501, 128)

In [14]:
model=models.efficientnet_b0()
for params in model.classifier.parameters():
    params.requires_grad=False
model.classifier=nn.Sequential(
    nn.Linear(1280,128),
    nn.ReLU(inplace=True),
    nn.Linear(128,20)

)
model.load_state_dict(torch.load('models/EffecientNet.h5'))

<All keys matched successfully>

In [15]:
model.classifier[1]=nn.Identity()
model.classifier[2]=nn.Identity()


In [16]:
effecientbo_train_feats,effecient_val_feats=feature_extractor(model,device,train_loader,valid_loader)

In [17]:
effecientbo_train_feats=np.array(effecientbo_train_feats)
effecient_val_feats=np.array(effecient_val_feats)
np.savez('effecient_train_feats',effecientbo_train_feats)
np.savez('effecient_val_feats',effecient_val_feats)

In [19]:
model = models.alexnet(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   



for param in model.parameters():
  param.requires_grad = False
model.classifier[4]=nn.Linear(4096,128)
model.classifier[6]=nn.Linear(128,20).to(device)
model.load_state_dict(torch.load('models/AlexNet.h5'))

d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [23]:
model.classifier[5]=nn.Identity()
model.classifier[6]=nn.Identity()
# new_model=nn.Sequential(*list(model.children())[:-1])
# new_model
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [24]:
alexnet_train_feats,alexnet_val_feats=feature_extractor(model,device,train_loader,valid_loader)

In [25]:
alexnet_train_feats=np.array(alexnet_train_feats)
alexnet_val_feats=np.array(alexnet_val_feats)
np.savez("alexnet_train_feats.npz",alexnet_train_feats)
np.savez("alexnet_val_feats.npz",alexnet_val_feats)

In [26]:
print(alexnet_val_feats.shape)

(2510, 128)


In [27]:
model=models.mobilenet_v2().to(device)
for param in model.classifier.parameters():
    param.requires_grad=False
model.classifier=nn.Sequential(
    nn.Linear(1280,128),
    nn.ReLU(inplace=True),
    nn.Linear(128,20)

)
model.load_state_dict(torch.load('models/MobileNet.h5'))


<All keys matched successfully>

In [28]:
model.classifier[1]=nn.Identity()
model.classifier[2]=nn.Identity()


In [29]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [30]:
mobilenet_train_feats,mobilenet_val_feats=feature_extractor(model,device,train_loader,valid_loader)

In [31]:
mobilenet_train_feats=np.array(mobilenet_train_feats)
mobilenet_val_feats=np.array(mobilenet_val_feats)
np.savez("mobilenet_train_feats.npz",mobilenet_train_feats)
np.savez("mobilenet_val_feats.npz",mobilenet_val_feats)

In [32]:
print(mobilenet_train_feats.shape)

(2501, 128)


In [38]:
model = models.inception_v3(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   

model.aux_logits = False

num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    nn.Linear(num_ftrs,128),
    nn.ReLU(inplace=True),
    nn.Linear(128,20)
)
model.load_state_dict(torch.load('models/Inception_V3.h5'))
model.fc[1]=nn.Identity()
model.fc[2]=nn.Identity()


d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
img_path='VOCdevkit-1/VOCdevkit/VOC2007/JPEGImages/'
train_img=[]
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
train_transformation = transforms.Compose([
                                      transforms.Resize((299, 299)),                                      
                                      transforms.RandomChoice([
                                          transforms.ColorJitter(brightness=(0.80, 1.20)),
                                          transforms.RandomGrayscale(p = 0.25)
                                          ]),
                                      transforms.RandomHorizontalFlip(p = 0.25),
                                      transforms.RandomRotation(25),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean = mean, std = std),
                                      ])
valid_transformation=transforms.Compose([transforms.Resize((299,299)), 
                                          transforms.ToTensor(), 
                                           transforms.Normalize(mean = mean, std = std),
                                          ])

data_dir='./VOCdevkit-1'
# Create train dataloader
dataset_train = PascalVOC_Dataset(data_dir,
                                      year='2007', 
                                      image_set='train', 
                                      download=False, 
                                      transform=train_transformation, 
                                      target_transform=encode_labels)
    
train_loader = DataLoader(dataset_train, batch_size=16, num_workers=0, shuffle=True)
dataset_valid = PascalVOC_Dataset(data_dir,
                                      year='2007', 
                                      image_set='val', 
                                      download=False, 
                                      transform=valid_transformation, 
                                      target_transform=encode_labels)
    
valid_loader = DataLoader(dataset_valid, batch_size=16, num_workers=0)

In [40]:
inceptionv3_train_feats,inceptionv3_val_feats=feature_extractor(model,device,train_loader,valid_loader)

In [41]:
inceptionv3_train_feats=np.array(inceptionv3_train_feats)
inceptionv3_val_feats=np.array(inceptionv3_val_feats)
np.savez("inceptionv3_train_feats.npz",inceptionv3_train_feats)
np.savez("inceptionv3_val_feats.npz",inceptionv3_val_feats)


In [45]:
print(inceptionv3_val_feats.shape)

(2510, 2048)


In [46]:
resnet50_train_feats=np.load("resnet50_train_feats.npz")
resnet50_val_feats=np.load("resnet50_val_feats.npz")
inceptionv3_train_feats=np.load("inceptionv3_train_feats.npz")
inceptionv3_val_feats=np.load("inceptionv3_val_feats.npz")
mobilenet_train_feats=np.load("mobilenet_train_feats.npz")
mobilenet_val_feats=np.load("mobilenet_val_feats.npz")
effecientbo_train_feats=np.load("effecient_train_feats.npz")
effecient_val_feats=np.load("effecient_val_feats.npz")
alexnet_train_feats=np.load("alexnet_train_feats.npz")
alexnet_val_feats=np.load("alexnet_val_feats.npz")

In [47]:
resnet50_train_feats=resnet50_train_feats['arr_0']
resnet50_val_feats=resnet50_val_feats['arr_0']
inceptionv3_train_feats=inceptionv3_train_feats['arr_0']
inceptionv3_val_feats=inceptionv3_val_feats['arr_0']
mobilenet_train_feats=mobilenet_train_feats['arr_0']
mobilenet_val_feats=mobilenet_val_feats['arr_0']
effecientbo_train_feats=effecientbo_train_feats['arr_0']
effecient_val_feats=effecient_val_feats['arr_0']
alexnet_train_feats=alexnet_train_feats['arr_0']
alexnet_val_feats=alexnet_val_feats['arr_0']
print(resnet50_train_feats[0].shape,inceptionv3_train_feats[0].shape,mobilenet_train_feats[0].shape,alexnet_train_feats[0].shape,effecientbo_train_feats[0].shape)

(128,) (128,) (128,) (128,) (128,)


In [48]:
concat_train_feats=[0]*2501
concat_val_feats=[0]*2510
for index in range(0,2501):
    concat_train_feats[index]=[]
    for arr in resnet50_train_feats[index]:
        concat_train_feats[index].append(arr)
for index in range(0,2510):
    concat_val_feats[index]=[]
    for arr in resnet50_val_feats[index]:
        concat_val_feats[index].append(arr)

      

In [50]:
print(len(concat_val_feats[0]))

128


In [51]:
for index in range(0,(2501)):
    for arr in inceptionv3_train_feats[index]:
        concat_train_feats[index].append(arr)
for index in range(0,2510):
    for arr in inceptionv3_val_feats[index]:
        concat_val_feats[index].append(arr)
  

In [52]:
for index in range(0,2501):
    for arr in mobilenet_train_feats[index]:
                concat_train_feats[index].append(arr)

In [53]:
for index in range(0,1000):
    for arr in mobilenet_val_feats[index]:
            concat_val_feats[index].append(arr)



In [54]:
for index in range(1000,2510):
    for arr in mobilenet_val_feats[index]:
                concat_val_feats[index].append(arr)

In [55]:
for index in range(0,2501):
    for arr in alexnet_train_feats[index]:
                concat_train_feats[index].append(arr)


In [56]:
for index in range(0,2510):
    for arr in alexnet_val_feats[index]:
                concat_val_feats[index].append(arr)    

In [57]:
for index in range(0,2501):
    for arr in effecientbo_train_feats[index]:
        concat_train_feats[index].append(arr)
for index in range(0,2510):
    for arr in effecient_val_feats[index]:
        concat_val_feats[index].append(arr)


In [59]:
train_labels=[]
for idx,(data,targets) in enumerate(train_loader):
    for t in targets:
        train_labels.append(t)
val_labels=[]
for idx,(data,targets) in enumerate(valid_loader):
    for t in targets:
     val_labels.append(t)
len(train_labels)


2501

In [60]:
for index in range(0,len(train_labels)):
    train_labels[index]=train_labels[index].cpu().detach().numpy()

In [62]:
for index in range(0,len(val_labels)):
    val_labels[index]=val_labels[index].cpu().detach().numpy()

In [63]:
import pandas as pd
from sklearn import svm
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multioutput import MultiOutputClassifier
svc=svm.SVC(probability=True,kernel='rbf')                                                                                                 
model=BinaryRelevance(svc)

In [64]:
df=pd.DataFrame(concat_train_feats)

In [65]:
train_labels=np.array(train_labels)

In [66]:
train_df=pd.concat([df,pd.DataFrame(train_labels)],axis=1)


In [67]:
train_df.shape

(2501, 660)

In [70]:
x=train_df.iloc[:,0:640]
y=train_df.iloc[:,640:660]

In [73]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x)


In [75]:
def splitIntoBatches(total_size,batch_size):
    if(total_size%batch_size==0):
        split =int(total_size/batch_size)
        return [batch_size*(i+1) for i in range(0,split)]
    else:
        list=[]
        split=int(total_size/batch_size)
        for i in range(0,split):
            list.append(batch_size*(i+1))
        list.append((split*(batch_size))+(total_size%batch_size))
        return list

In [74]:
model.fit(x,y)
 

BinaryRelevance(classifier=SVC(probability=True), require_dense=[True, True])

In [76]:
x_test=pd.DataFrame(concat_val_feats)

In [95]:

pred_prob=model.predict_proba(x_test)

In [106]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                     'cow', 'diningtable', 'dog', 'horse',
                     'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']

label="VOCdevkit-1/VOCdevkit/VOC2007/ImageSets/Main"

import os
img_Label_Dict=dict() 
temp=[]
for object in object_categories:
    for file in os.listdir(label):
        temp=[]
        name=str(object)+'_val.txt'
        if(file==name):
            file_obj=open(os.path.join(label,f"{file}"),"r")
            file_names=file_obj.read()
            lines=file_names.splitlines()
            for line in lines:
                content=line.split()
                if(('-1' or '0') not in content):
                    temp.append((content[0],content[1]))
        if(temp):
            img_Label_Dict[object]=temp



print(len(img_Label_Dict['aeroplane']))
  

127


In [107]:
from collections import defaultdict
correct_label_per_class=defaultdict(list)
for c in img_Label_Dict:
    for label in img_Label_Dict[c]:
        correct_label_per_class[c].append(int(label[1]))



In [109]:

file_names_list=0
dir='VOCdevkit-1/VOCdevkit/VOC2007/ImageSets/Main/'
image_dir='VOCdevkit-1/VOCdevkit/VOC2007/JPEGImages/'
for file in os.listdir(dir):
    if(file == f"val.txt"):
        #file_path=os.path.join(dir,"")
        file_obj=open(os.path.join(dir,f"{file}"),"r")
        file_names=file_obj.read()
        file_name_list=file_names.splitlines()        

        

In [110]:
counter=0
predicted_probs_per_class=defaultdict(list)
for c in img_Label_Dict:
    for label in img_Label_Dict[c]:
    
        for i,s in enumerate(file_name_list):
            if (str(label[0]) in s):
                predicted_probs_per_class[c].append(pred_prob.data[i][counter])
    counter=counter+1
            
              
            
           
                        

                    
       

In [111]:
from sklearn.metrics import average_precision_score
average_percision_per_class=[]
for c in object_categories:
    con_pred=[1 if x > 0.5 else 0 for x in predicted_probs_per_class[c]]
    average_percision_per_class.append(average_precision_score(correct_label_per_class[c],con_pred))
print(np.mean(average_percision_per_class))
                                

0.933405604574882
